For Running on Colab (Google Drive)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/gandhara-ish/gandhara-ish

Mounted at /content/drive
/content/drive/MyDrive/gandhara-ish/gandhara-ish
